In [22]:
import pandas as pd
import numpy as np

from flask import Flask, jsonify, render_template
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from pandas_geojson import to_geojson

In [23]:
engine = create_engine("sqlite:///sqlite3/melbournebusinessdatabase.db")
database_df = pd.read_sql("Select * from melbourne_business where census_year = 2020",engine)
database_df.head()

,Unnamed: 0,census_year,clue_small_area,trading_name,industry_anzsic4_description,seating_type,number_of_seats,x_coordinate,y_coordinate
0,50816,2020,Melbourne (CBD),551 Flinders Street MELBOURNE VIC 3000,Cafes and Restaurants,Seats - Indoor,60,144.9565145,-37.82097941
1,50817,2020,Melbourne (CBD),551 Flinders Street MELBOURNE VIC 3000,Cafes and Restaurants,Seats - Outdoor,6,144.9565145,-37.82097941
2,50818,2020,Melbourne (CBD),553 Flinders Street MELBOURNE VIC 3000,Takeaway Food Services,Seats - Indoor,12,144.9565145,-37.82097941
3,50819,2020,Melbourne (CBD),547 Flinders Street MELBOURNE VIC 3000,Cafes and Restaurants,Seats - Indoor,44,144.9565145,-37.82097941
4,50820,2020,Melbourne (CBD),563 Flinders Street MELBOURNE VIC 3000,Takeaway Food Services,Seats - Indoor,12,144.9559094,-37.82108687


In [24]:
database_df['number_of_seats'] = database_df['number_of_seats'].astype('int')
not_top7 = database_df.groupby('clue_small_area').sum().sort_values('number_of_seats',ascending = False).index[7:]
database_df = database_df.replace(not_top7, 'Other')
year1_df = database_df.groupby('clue_small_area', as_index=False).sum().sort_values('number_of_seats', ascending= False)

In [25]:
year1_df

,clue_small_area,number_of_seats
4,Melbourne (CBD),88974
1,Docklands,21585
7,Southbank,21215
0,Carlton,15177
6,Other,13196
5,Melbourne (Remainder),8767
2,East Melbourne,7181
3,Kensington,5709


In [26]:
database_df2 = pd.read_sql("Select * from melbourne_business where census_year = 2019",engine)
database_df2.head()

,Unnamed: 0,census_year,clue_small_area,trading_name,industry_anzsic4_description,seating_type,number_of_seats,x_coordinate,y_coordinate
0,47284,2019,Melbourne (CBD),551 Flinders Street MELBOURNE 3000,Cafes and Restaurants,Seats - Indoor,60,144.9565,-37.821
1,47285,2019,Melbourne (CBD),553 Flinders Street MELBOURNE 3000,Takeaway Food Services,Seats - Indoor,12,144.9565,-37.821
2,47286,2019,Melbourne (CBD),557-0 Flinders Street MELBOURNE 3000,Takeaway Food Services,Seats - Indoor,6,144.9565,-37.821
3,47287,2019,Melbourne (CBD),551 Flinders Street MELBOURNE 3000,Cafes and Restaurants,Seats - Outdoor,6,144.9565,-37.821
4,47288,2019,Melbourne (CBD),557-0 Flinders Street MELBOURNE 3000,Takeaway Food Services,Seats - Outdoor,4,144.9565,-37.821


In [27]:
database_df2['number_of_seats'] = database_df2['number_of_seats'].astype('int')
not_top7_2 = database_df2.groupby('clue_small_area').sum().sort_values('number_of_seats',ascending = False).index[7:]
database_df2 = database_df2.replace(not_top7_2, 'Other')
year2_df = database_df2.groupby('clue_small_area', as_index=False).sum().sort_values('number_of_seats', ascending= False)

In [28]:
year2_df

,clue_small_area,number_of_seats
4,Melbourne (CBD),96706
1,Docklands,22192
7,Southbank,21786
0,Carlton,15779
6,Other,14616
5,Melbourne (Remainder),8982
2,East Melbourne,7599
3,Kensington,5709


In [38]:
year1_df.compare(year2_df, keep_shape=True)

clue_small_area       number_of_seats         
             self other            self    other
4             NaN   NaN         88974.0  96706.0
1             NaN   NaN         21585.0  22192.0
7             NaN   NaN         21215.0  21786.0
0             NaN   NaN         15177.0  15779.0
6             NaN   NaN         13196.0  14616.0
5             NaN   NaN          8767.0   8982.0
2             NaN   NaN          7181.0   7599.0
3             NaN   NaN             NaN      NaN

In [39]:
year1_df["number_of_seats2"] = year2_df["number_of_seats"]
year1_df

,clue_small_area,number_of_seats,number_of_seats2
4,Melbourne (CBD),88974,96706
1,Docklands,21585,22192
7,Southbank,21215,21786
0,Carlton,15177,15779
6,Other,13196,14616
5,Melbourne (Remainder),8767,8982
2,East Melbourne,7181,7599
3,Kensington,5709,5709


In [45]:
year1_df["change"] = year1_df["number_of_seats"]-year1_df["number_of_seats2"]
year1_df["percent change"] =( year1_df["change"]/ year1_df["number_of_seats"]).map("{:.2%}".format)
year1_df

,clue_small_area,number_of_seats,number_of_seats2,change,percent change
4,Melbourne (CBD),88974,96706,-7732,-8.69%
1,Docklands,21585,22192,-607,-2.81%
7,Southbank,21215,21786,-571,-2.69%
0,Carlton,15177,15779,-602,-3.97%
6,Other,13196,14616,-1420,-10.76%
5,Melbourne (Remainder),8767,8982,-215,-2.45%
2,East Melbourne,7181,7599,-418,-5.82%
3,Kensington,5709,5709,0,0.00%
